In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

import os
import time
import tqdm
import pandas as pd
from copy import deepcopy
from typing import Dict

from sklearn.metrics import confusion_matrix

import math
import pickle

In [2]:
all_pat = pickle.load(open('all_pat.pkl', 'rb'))
all_visits = pickle.load(open('all_visits.pkl', 'rb'))
all_diags = pickle.load(open('all_diags.pkl', 'rb'))
mortality = pickle.load(open('mortality.pkl', 'rb'))
icd_dic = pickle.load(open('icd_dic.pkl', 'rb'))

In [3]:
print("Patient ID:", all_pat[3])
print("Dead:", mortality[3])
print("# of visits:", len(all_visits[3]))
for visit in range(len(all_visits[3])):
    print(f"\t{visit}-th visit id:", all_visits[3][visit])
    print(f"\t{visit}-th visit diagnosis labels:", all_diags[3][visit])

Patient ID: 252
Dead: 0
# of visits: 2
	0-th visit id: 190159
	0-th visit diagnosis labels: ['5712', '45620', '5780', '51881', '78559', '5722', '7895', '45341', '5990', '5849', '30500', '2867', '2761', '2875', '486', '5997', '8670', '2639', '2851', '5723', '34831', '2738', '5798', '0701', '3051', 'V1301', 'E8796']
	1-th visit id: 193470
	1-th visit diagnosis labels: ['5712', '45620', '7895', '2851', '3575']


In [4]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    
    def __init__(self, seqs, mortality):
        
        """
        TODO: Store `seqs`. to `self.x` and `hfs` to `self.y`.
        
        Note that you DO NOT need to covert them to tensor as we will do this later.
        Do NOT permute the data.
        """
        
        # your code here
        self.x = seqs
        self.y = mortality
#         raise NotImplementedError
    
    def __len__(self):
        
        """
        TODO: Return the number of samples (i.e. patients).
        """
        
        # your code here
        return len(self.y)
#         raise NotImplementedError
    
    def __getitem__(self, index):
        
        """
        TODO: Generates one sample of data.
        
        Note that you DO NOT need to covert them to tensor as we will do this later.
        """
        
        # your code here
        seq = self.x[index]
        hf = self.y[index]

        return seq, hf
#         raise NotImplementedError
        

dataset = CustomDataset(all_diags, mortality)

In [5]:
num_visits = [len(patient) for patient in all_visits]
num_codes = [len(visit) for patient in all_diags for visit in patient]
max_num_visits = max(num_visits)
max_num_codes = max(num_codes)
max_num_visits, max_num_codes

(42, 39)

In [15]:
def collate_fn(data):
    
    sequences, labels = zip(*data)
    y = torch.tensor(labels, dtype=torch.int64)
    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]
    max_num_visits = 42 #max(num_visits)
    max_num_codes = 39 #max(num_codes)
#     print(max_num_visits, max_num_codes, len(y) )
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.float)
    for i_patient, patient in enumerate(sequences):
        for j_visit, visit in enumerate(patient):
            for k_diag, diag in enumerate(visit):
                if diag != diag:
#                     print(diag, i_patient, j_visit,k_diag) 
                    continue
                else: x[i_patient, j_visit,k_diag ] =icd_dic[diag]
#     x = x.type(torch.LongTensor)
#     y = y.type(torch.LongTensor)
    return x, y

In [16]:
from torch.utils.data.dataset import random_split

split = int(len(dataset)*0.7)

lengths = [split, len(dataset) - split]
train_dataset, val_test_dataset = random_split(dataset, lengths)


split = int(len(val_test_dataset)*0.7)
lengths = [split, len(val_test_dataset) - split]
val_dataset, test_dataset = random_split(val_test_dataset, lengths)


print("Length of train dataset:", len(train_dataset))
print("Length of val dataset:", len(val_dataset))
print("Length of test dataset:", len(test_dataset))

Length of train dataset: 32563
Length of val dataset: 9769
Length of test dataset: 4188


In [17]:
def load_data(train_dataset, val_dataset, test_dataset, batch_size, collate_fn):
    batch_size = batch_size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)
    
    return train_loader, val_loader, test_loader

batch_size = 128
train_loader, val_loader, test_loader = load_data(train_dataset, val_dataset, test_dataset, batch_size, collate_fn)

in_feature = 39
seq_len = 42
n_heads = 32
factor = 5 # M - dense interpolation factor
num_class = 2
num_layers = 1 #N


In [18]:
for i in train_loader:
    print(i[0].shape)

torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([128, 42, 39])
torch.Size([

In [86]:
class MyNeuralNetworkClassifier:
    def __init__(self, model, criterion, optimizer, optimizer_config: dict) -> None:
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)
        self.optimizer = optimizer(self.model.parameters(), **optimizer_config)
        self.criterion = criterion

        self.hyper_params = optimizer_config
        self._start_epoch = 0
        self.hyper_params["epochs"] = self._start_epoch
        self.__num_classes = None
        self._is_parallel = False

        if torch.cuda.device_count() > 1:
            self.model = nn.DataParallel(self.model)
            self._is_parallel = True

            notice = "Running on {} GPUs.".format(torch.cuda.device_count())
            print("\033[33m" + notice + "\033[0m")
            
    def fit(self, loader: Dict[str, DataLoader], epochs: int, checkpoint_path: str = None, validation: bool = True) -> None:
        len_of_train_dataset = len(loader["train"].dataset)
        epochs = epochs + self._start_epoch

        self.hyper_params["epochs"] = epochs
        self.hyper_params["batch_size"] = loader["train"].batch_size
        self.hyper_params["train_ds_size"] = len_of_train_dataset
        
       
        
        if validation:
            len_of_val_dataset = len(loader["val"].dataset)
            self.hyper_params["val_ds_size"] = len_of_val_dataset
            
        for epoch in range(self._start_epoch, epochs):
            if checkpoint_path is not None and epoch % 100 == 0:
                self.save_to_file(checkpoint_path)
            correct = 0.0
            total = 0.0
            
            self.model.train()
            pbar = tqdm.tqdm(total=len_of_train_dataset)
            for x, y in loader["train"]:
#                 print(x.shape, y.shape)
                b_size = y.shape[0]
                total += y.shape[0]
                x = x.to(self.device) if isinstance(x, torch.Tensor) else [i.to(self.device) for i in x]
                y = y.to(self.device)

                pbar.set_description(
                    "\033[36m" + "Training" + "\033[0m" + " - Epochs: {:03d}/{:03d}".format(epoch+1, epochs)
                )
                pbar.update(b_size)

                self.optimizer.zero_grad()
#                 print("Here1")
                outputs = self.model(x)
#                 print("Here2")
                loss = self.criterion(outputs, y)
                loss.backward()
                self.optimizer.step()

                _, predicted = torch.max(outputs, 1)
                correct += (predicted == y).sum().float().cpu().item()

                
    def evaluate(self, loader: DataLoader, verbose: bool = False) -> None or float:

        running_loss = 0.0
        running_corrects = 0.0
        pbar = tqdm.tqdm(total=len(loader.dataset))

        self.model.eval()
        print("test_ds_size", len(loader.dataset))



        with torch.no_grad():
            correct = 0.0
            total = 0.0
#                 for x, y in enumerate(loader): SAYAN
            for x, y in loader:
#                 print("Here is type & shape", type(y), y.shape)
                b_size = y.shape[0]
                total += y.shape[0]
                x = x.to(self.device) if isinstance(x, torch.Tensor) else [i.to(self.device) for i in x]
                y = y.to(self.device)

                pbar.set_description("\033[32m"+"Evaluating"+"\033[0m")
                pbar.update(b_size)

                outputs = self.model(x)
                loss = self.criterion(outputs, y)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == y).sum().float().cpu().item()

                running_loss += loss.cpu().item()
                running_corrects += torch.sum(predicted == y).float().cpu().item()
                print("loss", running_loss)
                acc = float(running_corrects / total)
                print("accuracy", acc)
#                 experiment.log_metric("Accuracy", acc)
                
            pbar.close()
#         acc = self.experiment.get_metric("accuracy")
        

        print("\033[33m" + "Evaluation finished. " + "\033[0m" + "Accuracy: {:.4f}".format(acc))

        if verbose:
            return acc

 
    def save_checkpoint(self) -> dict:
        """
        The method of saving trained PyTorch model.

        Note,  return value contains
            - the number of last epoch as `epochs`
            - optimizer state as `optimizer_state_dict`
            - model state as `model_state_dict`

        ::

            clf = NeuralNetworkClassifier(
                    Network(), nn.CrossEntropyLoss(),
                    optim.Adam, optimizer_config, experiment
                )

            clf.fit(train_loader, epochs=10)
            checkpoints = clf.save_checkpoint()

        :return: dict {'epoch', 'optimizer_state_dict', 'model_state_dict'}
        """

        checkpoints = {
            "epoch": deepcopy(self.hyper_params["epochs"]),
            "optimizer_state_dict": deepcopy(self.optimizer.state_dict())
        }

        if self._is_parallel:
            checkpoints["model_state_dict"] = deepcopy(self.model.module.state_dict())
        else:
            checkpoints["model_state_dict"] = deepcopy(self.model.state_dict())

        return checkpoints

    def save_to_file(self, path: str) -> str:
        """
        | The method of saving trained PyTorch model to file.
        | Those weights are uploaded to comet.ml as backup.
        | check "Asserts".

        Note, .pth file contains
            - the number of last epoch as `epochs`
            - optimizer state as `optimizer_state_dict`
            - model state as `model_state_dict`

        ::

            clf = NeuralNetworkClassifier(
                    Network(), nn.CrossEntropyLoss(),
                    optim.Adam, optimizer_config, experiment
                )

            clf.fit(train_loader, epochs=10)
            filename = clf.save_to_file('path/to/save/dir/')

        :param path: path to saving directory. : string
        :return: path to file : string
        """
        if not os.path.isdir(path):
            os.mkdir(path)

        file_name = "model_params-epochs_{}-{}.pth".format(
            self.hyper_params["epochs"], time.ctime().replace(" ", "_")
        )
        path = path + file_name

        checkpoints = self.save_checkpoint()

        torch.save(checkpoints, path)
#         self.experiment.log_asset(path, file_name=file_name)

        return path

In [87]:

class PositionWiseFeedForward(nn.Module):
    def __init__(self, hidden_size: int) -> None:
        super(PositionWiseFeedForward, self).__init__()
        self.hidden_size = hidden_size

        self.conv = nn.Sequential(
            nn.Conv1d(hidden_size, hidden_size * 2, 1),
            nn.ReLU(),
            nn.Conv1d(hidden_size * 2, hidden_size, 1)
        )

    def forward(self, tensor: torch.Tensor) -> torch.Tensor:
        tensor = tensor.transpose(1, 2)
        tensor = self.conv(tensor)
        tensor = tensor.transpose(1, 2)

        return tensor




class PositionalEncoding(nn.Module):
    def __init__(self, d_model, seq_len) -> None:
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model

        pe = torch.zeros(seq_len, d_model)

        for pos in range(seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i) / d_model)))
                pe[pos, i+1] = math.cos(pos / (10000 ** ((2 * (i+1)) / d_model)))

        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x) -> torch.Tensor:
        seq_len = x.shape[1]
        x = math.sqrt(self.d_model) * x
        x = x + self.pe[:, :seq_len].requires_grad_(False)
        return x

class EncoderBlock(nn.Module):
    def __init__(self, embed_dim: int, num_head: int, dropout_rate=0.1) -> None:
        super(EncoderBlock, self).__init__()
        self.attention = ResidualBlock(
            nn.MultiheadAttention(embed_dim, num_head), embed_dim, p=dropout_rate
        )
        self.ffn = ResidualBlock(PositionWiseFeedForward(embed_dim), embed_dim, p=dropout_rate)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
#         print("EncoderBlock", x.shape)
        x = self.attention(x)
        x = self.ffn(x)
        return x



class ResidualBlock(nn.Module):
    def __init__(self, layer: nn.Module, embed_dim: int, p=0.1) -> None:
        super(ResidualBlock, self).__init__()
        self.layer = layer
        self.dropout = nn.Dropout(p=p)
        self.norm = nn.LayerNorm(embed_dim)
        self.attn_weights = None

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        :param x: [N, seq_len, features]
        :return: [N, seq_len, features]
        """
        if isinstance(self.layer, nn.MultiheadAttention):
            src = x.transpose(0, 1)     # [seq_len, N, features]
            output, self.attn_weights = self.layer(src, src, src)
            output = output.transpose(0, 1)     # [N, seq_len, features]

        else:
            output = self.layer(x)

        output = self.dropout(output)
        output = self.norm(x + output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, seq_len) -> None:
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model

        pe = torch.zeros(seq_len, d_model)

        for pos in range(seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i) / d_model)))
                pe[pos, i+1] = math.cos(pos / (10000 ** ((2 * (i+1)) / d_model)))

        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x) -> torch.Tensor:
        seq_len = x.shape[1]
        x = math.sqrt(self.d_model) * x
        x = x + self.pe[:, :seq_len].requires_grad_(False)
        return x 

class EncoderLayerForSAnD(nn.Module):
    def __init__(self, input_features, seq_len, n_heads, n_layers, d_model=128, dropout_rate=0.2) -> None:
        super(EncoderLayerForSAnD, self).__init__()
        self.d_model = d_model

        self.input_embedding = nn.Conv1d(input_features, d_model, 1)
        self.positional_encoding = PositionalEncoding(d_model, seq_len)
        self.blocks = nn.ModuleList([
            EncoderBlock(d_model, n_heads, dropout_rate) for _ in range(n_layers)
        ])

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        x = x.transpose(1, 2)
#         print(x.type())
        x = self.input_embedding(x)
        x = x.transpose(1, 2)

        x = self.positional_encoding(x)

        for l in self.blocks:
            x = l(x)

        return x

class DenseInterpolation(nn.Module):
    def __init__(self, seq_len: int, factor: int) -> None:
        """
        :param seq_len: sequence length
        :param factor: factor M
        """
        super(DenseInterpolation, self).__init__()

        W = np.zeros((factor, seq_len), dtype=np.float32)

        for t in range(seq_len):
            s = np.array((factor * (t + 1)) / seq_len, dtype=np.float32)
            for m in range(factor):
                tmp = np.array(1 - (np.abs(s - (1+m)) / factor), dtype=np.float32)
                w = np.power(tmp, 2, dtype=np.float32)
                W[m, t] = w

        W = torch.tensor(W).float().unsqueeze(0)
        self.register_buffer("W", W)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        w = self.W.repeat(x.shape[0], 1, 1).requires_grad_(False)
        u = torch.bmm(w, x)
        return u.transpose_(1, 2)


class ClassificationModule(nn.Module):
    def __init__(self, d_model: int, factor: int, num_class: int) -> None:
        super(ClassificationModule, self).__init__()
        self.d_model = d_model
        self.factor = factor
        self.num_class = num_class

        self.fc = nn.Linear(int(d_model * 42), num_class)
#         self.fc = nn.Linear(int(d_model * factor), num_class)
        self.sm = nn.Softmax()

        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        a,b,c = x.shape
        print("Shape",a,b,c)
        x = x.contiguous().view(-1, int(42 * self.d_model))
#         x = self.fc0(x)
        print("new shape",x.shape)
        x = self.fc(x)
        return x

    

class RegressionModule(nn.Module):
    def __init__(self, d_model: int, factor: int, output_size: int) -> None:
        super(RegressionModule, self).__init__()
        self.d_model = d_model
        self.factor = factor
        self.output_size = output_size
        self.fc = nn.Linear(int(d_model * factor), output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.contiguous().view(-1, int(self.factor * self.d_model))
        x = self.fc(x)
        return x


In [88]:
class SAnD(nn.Module):

    def __init__(
            self, input_features: int, seq_len: int, n_heads: int, factor: int,
            n_class: int, n_layers: int, d_model: int = 128, dropout_rate: float = 0.2
    ) -> None:
        super(SAnD, self).__init__()
        self.encoder = EncoderLayerForSAnD(input_features, seq_len, n_heads, n_layers, d_model, dropout_rate)
        self.dense_interpolation = DenseInterpolation(seq_len, factor)
        self.clf = ClassificationModule(d_model, factor, n_class)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.encoder(x)
#         x = self.dense_interpolation(x)
        x = self.clf(x)
        return x

In [89]:
from comet_ml import Experiment

# Initialize Comet.ml with your API key
experiment = Experiment(api_key="dGXMo2OT0jVdMZPp2KVui30tD", project_name="general")

clf = MyNeuralNetworkClassifier(
    SAnD(in_feature, seq_len, n_heads, factor, num_class, num_layers),
    nn.CrossEntropyLoss(),
    optim.Adam, 
    optimizer_config={"lr": 1e-5, "betas": (0.9, 0.98), "eps": 4e-09, "weight_decay": 5e-4}
    )


COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch, sklearn. Metrics and hyperparameters can still be logged using Experiment.log_metrics() and Experiment.log_parameters()
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/sayan3/general/821352373a334c489b758a92af7b5073
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     source_code         : 1
COMET INFO: ---------------------------
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch, sklearn. Metrics and hyperparameters can still be logged using Experiment.log

In [96]:
# training network
clf.fit(
    {"train": train_loader,
     "val": val_loader},
     epochs= 5
)












  0%|                                                                            | 0/32563 [00:00<?, ?it/s]









Training - Epochs: 001/005:   0%|                                                | 0/32563 [00:00<?, ?it/s]









Training - Epochs: 001/005:   0%|▏                                   | 128/32563 [00:00<00:30, 1063.18it/s]









Training - Epochs: 001/005:   1%|▎                                   | 256/32563 [00:00<00:15, 2109.98it/s]









Training - Epochs: 001/005:   1%|▎                                   | 256/32563 [00:00<00:15, 2109.98it/s]

Shape 128 42 128
new shape torch.Size([128, 5376])
Shape 128 42 128
new shape torch.Size([128, 5376])












Training - Epochs: 001/005:   1%|▍                                   | 384/32563 [00:00<00:15, 2109.98it/s]









Training - Epochs: 001/005:   2%|▌                                   | 512/32563 [00:00<00:22, 1408.39it/s]









Training - Epochs: 001/005:   2%|▌                                   | 512/32563 [00:00<00:22, 1408.39it/s]

Shape 128 42 128
new shape torch.Size([128, 5376])
Shape 128 42 128
new shape torch.Size([128, 5376])












Training - Epochs: 001/005:   2%|▋                                   | 640/32563 [00:00<00:22, 1408.39it/s]









Training - Epochs: 001/005:   2%|▊                                   | 768/32563 [00:00<00:24, 1284.76it/s]









Training - Epochs: 001/005:   2%|▊                                   | 768/32563 [00:00<00:24, 1284.76it/s]

...

Shape 128 42 128
new shape torch.Size([128, 5376])
Shape 128 42 128
new shape torch.Size([128, 5376])












Training - Epochs: 005/005:  99%|█████████████████████████████████▊| 32384/32563 [00:27<00:00, 1150.74it/s]









Training - Epochs: 005/005: 100%|█████████████████████████████████▉| 32512/32563 [00:27<00:00, 1151.85it/s]









Training - Epochs: 005/005: 100%|█████████████████████████████████▉| 32512/32563 [00:27<00:00, 1151.85it/s]

Shape 128 42 128
new shape torch.Size([128, 5376])
Shape 128 42 128
new shape torch.Size([128, 5376])


Training - Epochs: 005/005: 100%|██████████████████████████████████| 32563/32563 [00:27<00:00, 1171.79it/s]

Shape 51 42 128
new shape torch.Size([51, 5376])


In [97]:
clf.evaluate(test_loader)











  0%|                                                                             | 0/4188 [00:00<?, ?it/s]









Evaluating:   0%|                                                                 | 0/4188 [00:00<?, ?it/s]









Evaluating:   3%|█▌                                                   | 128/4188 [00:00<00:01, 2137.15it/s]









Evaluating:   6%|███▏                                                 | 256/4188 [00:00<00:01, 2404.17it/s]









Evaluating:   9%|████▊                                                | 384/4188 [00:00<00:01, 3569.04it/s]

test_ds_size 4188
Shape 128 42 128
new shape torch.Size([128, 5376])
loss 0.5038946866989136
accuracy 0.828125
Shape 128 42 128
new shape torch.Size([128, 5376])
loss 0.9246344864368439
accuracy 0.8359375
Shape 128 42 128
new shape torch.Size([128, 5376])












Evaluating:   9%|████▊                                                | 384/4188 [00:00<00:01, 3569.04it/s]

loss 1.2270499169826508
accuracy 0.8489583333333334
Shape 128 42 128
new shape torch.Size([128, 5376])
loss 1.586095005273819
accuracy 0.853515625












Evaluating:  12%|██████▍                                              | 512/4188 [00:00<00:01, 3569.04it/s]









Evaluating:  15%|████████                                             | 640/4188 [00:00<00:00, 3569.04it/s]









Evaluating:  18%|█████████▋                                           | 768/4188 [00:00<00:01, 2814.32it/s]









Evaluating:  18%|█████████▋                                           | 768/4188 [00:00<00:01, 2814.32it/s]

...

Shape 128 42 128
new shape torch.Size([128, 5376])
loss 10.746214777231216
accuracy 0.8616898148148148
Shape 128 42 128
new shape torch.Size([128, 5376])
loss 11.293012410402298
accuracy 0.8602120535714286
Shape 128 42 128
new shape torch.Size([128, 5376])
loss 11.707283765077591
accuracy 0.859375












Evaluating:  89%|██████████████████████████████████████████████      | 3712/4188 [00:01<00:00, 2660.52it/s]









Evaluating:  92%|███████████████████████████████████████████████▋    | 3840/4188 [00:01<00:00, 2698.57it/s]









Evaluating:  92%|███████████████████████████████████████████████▋    | 3840/4188 [00:01<00:00, 2698.57it/s]

Shape 128 42 128
new shape torch.Size([128, 5376])
loss 12.171696424484253
accuracy 0.8588541666666667
Shape 128 42 128
new shape torch.Size([128, 5376])
loss 12.5442413687706
accuracy 0.859375












Evaluating:  95%|█████████████████████████████████████████████████▎  | 3968/4188 [00:01<00:00, 2698.57it/s]









Evaluating:  98%|██████████████████████████████████████████████████▊ | 4096/4188 [00:01<00:00, 2698.57it/s]









Evaluating: 100%|████████████████████████████████████████████████████| 4188/4188 [00:01<00:00, 2668.23it/s]

Shape 128 42 128
new shape torch.Size([128, 5376])
loss 13.05088722705841
accuracy 0.857666015625
Shape 92 42 128
new shape torch.Size([92, 5376])
loss 13.445675998926163
accuracy 0.8574498567335244
Evaluation finished. Accuracy: 0.8574


In [98]:


correct = 0.0
total = 0.0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pbar = tqdm.tqdm(total=len(test_loader.dataset))
model = SAnD(in_feature, seq_len, n_heads, factor, num_class, num_layers)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
Y_pred= []
Y_true = []
for x, y in test_loader:
#     print("Here is type & shape", type(y), y.shape)
    b_size = y.shape[0]
    total += y.shape[0]
    x = x.to(device) if isinstance(x, torch.Tensor) else [i.to(device) for i in x]
    y = y.to(device)
    
    pbar.set_description("\033[32m"+"Evaluating"+"\033[0m")
    pbar.update(b_size)

    outputs = model(x)
#     print(outputs)
    idx = torch.argmax(outputs, dim=1) 
    print(idx)
#     _, predicted = torch.max(outputs, 1)
    Y_pred.append(idx)
    Y_true.append(y)
    yp = np.concatenate(Y_pred, axis=0)
    yt = np.concatenate(Y_true, axis=0)
    roc = roc_auc_score(
                yt,
                yp
        )
    experiment.log_metric("ROC", roc)

#     multilabel_auroc(predicted, y,num_labels=2, average="macro", thresholds=None)

Y_pred = np.concatenate(Y_pred, axis=0)
Y_true = np.concatenate(Y_true, axis=0)











Evaluating: 100%|██████████████████████████████████████████████████████| 4188/4188 [06:05<00:00, 11.46it/s][A










Evaluating:   0%|                                                                 | 0/4188 [00:00<?, ?it/s]









Evaluating:   3%|█▌                                                   | 128/4188 [00:00<00:03, 1199.58it/s]









Evaluating:   6%|███▏                                                 | 256/4188 [00:00<00:01, 2378.83it/s]

Shape 128 42 128
new shape torch.Size([128, 5376])
tensor([1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
        0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1,
        1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        0, 1, 0, 0, 1, 0, 1, 1])
Shape 128 42 128
new shape torch.Size([128, 5376])
tensor([1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1,
        1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
        1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
        1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1])












Evaluating:   6%|███▏                                                 | 256/4188 [00:00<00:01, 2378.83it/s]









Evaluating:   9%|████▊                                                | 384/4188 [00:00<00:01, 2378.83it/s]









Evaluating:  12%|██████▍                                              | 512/4188 [00:00<00:01, 2478.93it/s]









Evaluating:  12%|██████▍                                              | 512/4188 [00:00<00:01, 2478.93it/s]

Shape 128 42 128
new shape torch.Size([128, 5376])
tensor([0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 0, 1, 1, 1])
Shape 128 42 128
new shape torch.Size([128, 5376])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0,
        1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
        1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
        1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
        1, 0, 1, 1, 0, 1, 1, 1])












Evaluating:  15%|████████                                             | 640/4188 [00:00<00:01, 2478.93it/s]









Evaluating:  18%|█████████▋                                           | 768/4188 [00:00<00:01, 2386.63it/s]









Evaluating:  18%|█████████▋                                           | 768/4188 [00:00<00:01, 2386.63it/s]

...

Shape 128 42 128
new shape torch.Size([128, 5376])
tensor([1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
        1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1])
Shape 128 42 128
new shape torch.Size([128, 5376])
tensor([1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
        1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
        0, 1, 1, 1, 1, 0, 1, 1])












Evaluating:  92%|███████████████████████████████████████████████▋    | 3840/4188 [00:01<00:00, 1987.58it/s]

Shape 128 42 128
new shape torch.Size([128, 5376])
tensor([0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0,
        1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
        0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 1, 1])












Evaluating:  95%|█████████████████████████████████████████████████▎  | 3968/4188 [00:01<00:00, 1987.58it/s]









Evaluating:  98%|██████████████████████████████████████████████████▊ | 4096/4188 [00:01<00:00, 1988.16it/s]









Evaluating:  98%|██████████████████████████████████████████████████▊ | 4096/4188 [00:01<00:00, 1988.16it/s]

Shape 128 42 128
new shape torch.Size([128, 5376])
tensor([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 0])
Shape 128 42 128
new shape torch.Size([128, 5376])
tensor([1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
        1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
        0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 0, 0, 1, 1])
Shape 92 42 128
new shape torch.

In [101]:
from sklearn.metrics import roc_auc_score
roc_auc_score(
    Y_true,
    Y_pred
)

0.4842837521438687

In [100]:
from sklearn.metrics import average_precision_score

# pred_np = Y_pred.detach().cpu().numpy()
# gt_labels_np = Y_true.detach().cpu().numpy()
auprc_score = average_precision_score(Y_true, Y_pred)
auprc_score

0.1360508381454862